In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils.plotting as plotting
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime
from model.model_fast import Fast

# Load Data from File

In [ ]:
folder_path = os.path.join('output_data', 'fast_run_data')
files = os.listdir(folder_path)

# OPTIONAL Filter files that start with 'results_batch_'
files = [file for file in files if file.startswith('results_batch_')]

# Get the most recently modified file path
most_recent_file = max(files, key=lambda x: os.path.getmtime(os.path.join(folder_path, x)))
most_recent_file_path = os.path.join(folder_path, most_recent_file)
# print(most_recent_file_path)

df = pd.read_csv(most_recent_file_path)


# Generate Data with Batch Run

In [ ]:
from mesa.batchrunner import batch_run
batch_parameters = {
    'data_collection_period': 5,
    'iterations': 1,
    'max_steps': 200
}

# Define the variable and fixed parameters
variable_parameters = {
    'density': [1, 100],
    # 'subsistence_wage': [10000, 30000],
    # 'gamma': [0.02]
}

# Define the variable and fixed parameters
fixed_parameters = {
            'run_notes': 'Debugging model.',
            'subfolder': None,
            'width':     20,
            'height':    20,

            # FLAGS
            'demographics_on': True,  # Set flag to False for debugging to check firm behaviour without demographics or housing market
            'center_city':     False, # Flag for city center in center if True, or bottom corner if False
            # 'random_init_age': False,  # Flag for randomizing initial age. If False, all workers begin at age 0
            'random_init_age': True,  # Flag for randomizing initial age. If False, all workers begin at age 0

            # LABOUR MARKET AND FIRM PARAMETERS
            'subsistence_wage': 40000., # psi
            'init_city_extent': 10.,    # CUT OR CHANGE?
            'seed_population': 400,
            'init_wage_premium_ratio': 0.2, # 1.2, ###

            # PARAMETERS MOST LIKELY TO AFFECT SCALE
            'c': 300.0,                            ###
            'price_of_output': 10,                 ######
            'density': 600,                         #####
            'A': 3000,                             ### 
            'alpha': 0.18,
            'beta':  0.75,
            'gamma': 0.12, ### reduced from .14
            'overhead': 1,
            'mult': 1.2,
            'adjN': 0.15,
            'adjk': 0.10,
            'adjn': 0.25,
            'adjF': 0.15,
            'adjw': 0.02, 
            'dist': 1, 
            'init_F': 100.0,
            'init_k': 500.0,
            'init_n': 100.0,

            # HOUSING AND MORTGAGE MARKET PARAMETERS
            'mortgage_period': 5.0,       # T, in years
            'working_periods': 40,        # in years
            'savings_rate': 0.3,
            'discount_rate': 0.07,        # 1/delta
            'r_prime': 0.05,
            'r_margin': 0.01,
            'r_investor': 0.05,            # Next best alternative return for investor
            'property_tax_rate': 0.04,     # tau, annual rate, was c
            'housing_services_share': 0.3, # a
            'maintenance_share': 0.2,      # b
            'max_mortgage_share': 0.9,
            'ability_to_carry_mortgage': 0.28,
            'wealth_sensitivity': 0.1,
            'cg_tax_per':   0.0, # share 0-1
            'cg_tax_invest': 0.1, # share 0-1
        }

model_parameters = {**fixed_parameters, **variable_parameters} # OR model_parameters = variable_parameters

results = batch_run(Fast, model_parameters, **batch_parameters)
df = pd.DataFrame(results)

subfolder = os.path.join('output_data', 'fast_run_data')
os.makedirs(subfolder, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d-%H%M%S")
filepath = os.path.join(subfolder, f'results_batch_{timestamp}.csv')

df.to_csv(filepath, index=False)

# Plot Batch Output as Small Multiples

In [ ]:
filtered_df = df[df['bidder_name'].notna()]
bidder_categories = filtered_df['bidder_name'].unique().tolist()
color_palette = plotting.get_bidder_color_palette(bidder_categories)
print(bidder_categories)
print(color_palette)

In [ ]:
# Select only bid storage data
filtered_df = df.loc[df['agent_type'] == 'Bid_Storage']

# # Select only data where other parameters are fixed
# filtered_df = filtered_df[filtered_df['subsistence_wage'] == 10000] #filtered_df.loc[filtered_df['subsistence_wage'] == 1000]
# # print(filtered_df)

# # Remove data before the start_time
# start_time = 10
# filtered_df = filtered_df[filtered_df['time_step'] >= start_time]

# # Downsample time_steps as appropriate
# filtered_df = plotting.downsample(df = filtered_df, var = 'time_step', no_vals_to_plot = 4)
    
parameter_mapping = {
    'x': 'time_step',
    'y': 'bid',
    'x_global': 'distance', #'density', #'Distance', # 'param_1',
    'y_global': 'density',
    'line_val': 'bidder_name'
}

plotting.small_multiples_lineplot(filtered_df, parameter_mapping, color_palette)

# Single Run

In [ ]:
# Run model
num_steps  = 10
parameters = {
            'width':     3,
            'height':    3,
}
city = Fast(num_steps, **parameters)
for time_step in range(num_steps): # TODO time here and above
    # print(f'Step {time}')
    city.step()
city.record_run_data_to_file()
agent_out = city.datacollector.get_agent_vars_dataframe()
model_out = city.datacollector.get_model_vars_dataframe()

### This is an outdated way of plotting based on the old formulation of data as strings

In [ ]:
# Plot 
# Assuming model_out['R_N'], model_out['investor_bid'], etc., are pandas Series
attributes_to_plot = ['n', 'y', 'F_target', 'F', 'k', 'N']  # ["R_N", "investor_bid", "warranted_rent", "warranted_price", "maintenance"]

# Create subplots
num_rows = len(attributes_to_plot)
num_cols = 1  # You can adjust the number of columns as needed
fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, 6 * num_rows))



# first_elements = [lst[0] for lst in model_out['R_N']]

# Loop over attributes
for i, attribute in enumerate(attributes_to_plot):
    # Get the values for the current attribute
    values_by_position = np.array([lst for lst in model_out[attribute]])

    # Plot each value as a point against time_step
    for position in range(values_by_position.shape[1]):
        values = values_by_position[:, position]
        axs[i].scatter(model_out['time_step'], values, label=f"Position {position + 1}")

        # Connect points with lines
        axs[i].plot(model_out['time_step'], values, linestyle='--')

    axs[i].set_xlabel('Time Step')
    axs[i].set_ylabel(f'Values from {attribute}')
    axs[i].legend()

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming model_out['R_N'] is a pandas Series
values_by_position = np.array([lst for lst in model_out['R_N']])

# Plotting
num_positions = values_by_position.shape[1]

# Get distinct colors for each position
colors = plt.cm.viridis(np.linspace(0, 1, num_positions))

# Create subplots
fig, ax = plt.subplots(figsize=(10, 6))

# Plot each value as a point against time_step
for position in range(num_positions):
    values = values_by_position[:, position]
    ax.scatter(model_out['time_step'], values, label=f"Position {position + 1}", color=colors[position])

    # Connect points with lines
    ax.plot(model_out['time_step'], values, color=colors[position], linestyle='--')

ax.set_xlabel('Time Step')
ax.set_ylabel('Values from R_N')
ax.legend()
plt.show()


In [ ]:
# Extract attributes for plotting (excluding 'dist')
attributes_to_plot = ["R_N", "investor_bid", "warranted_rent", "warranted_price", "maintenance"]

# Create subplots
num_rows = 4
num_cols = 2
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15))
fig.suptitle("Attributes vs Distance at Each Time Step", y=1.02, fontsize='x-large')  # Increased font size for the title

# Get distinct colors for each time step
num_time_steps = city.num_steps
colors = plt.cm.viridis(np.linspace(0, 1, num_time_steps))

# Loop over attributes
for i, attribute in enumerate(attributes_to_plot):
    # Loop over time steps
    for time_step in range(2, num_time_steps + 1):
        # Load data for the current time step
        loaded_data = model_out[model_out['time_step'] == time_step]

        # Extract all values from the list for each time step
        investor_bid_values = loaded_data[attribute].apply(
            lambda x: x if isinstance(x, list) and len(x) > 0 else [x]
        )

        # Plot all values on the same plot
        for j, bids in enumerate(investor_bid_values.tolist()):
            axs[i // num_cols, i % num_cols].plot(
                [time_step] * len(bids),
                bids,
                marker='o',
                label=f"{attribute} - Time Step {time_step}",
                color=colors[time_step - 1]
            )

    axs[i // num_cols, i % num_cols].set_title(attribute, fontsize='large')  # Increased font size for the title
    axs[i // num_cols, i % num_cols].set_xlabel("Time Step", fontsize='medium')  # Increased font size for x-axis label
    axs[i // num_cols, i % num_cols].set_ylabel(attribute, fontsize='medium')  # Increased font size for y-axis label

# Add a legend to the last subplot
legend_ax = axs[-1, -1]
handles, labels = axs[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, title='Time Steps', bbox_to_anchor=(.9, .10), loc='lower right', fontsize='large')
legend_ax.axis('off')  # Disable axis for the legend

plt.tight_layout()
plt.show()


In [ ]:
# Extract attributes for plotting
attributes_to_plot = ["city_extent_calc"]

# Create subplots
num_rows = 1
num_cols = 1
fig, axs = plt.subplots(num_rows, num_cols, figsize=(8, 6))
fig.suptitle("City Extent Calculation Over Time", y=1.02, fontsize='x-large')  # Increased font size for the title

# Load data
loaded_data = model_out

# Plot each attribute
for attribute in attributes_to_plot:
    axs.plot(
        loaded_data['time_step'],
        loaded_data[attribute],
        label=attribute
    )
    axs.set_xlabel("Time Step", fontsize='medium')  # Increased font size for x-axis label
    axs.set_ylabel("City Extent Calculation", fontsize='medium')  # Increased font size for y-axis label

# Add a legend
axs.legend(title='Attributes', loc='upper left', fontsize='large')

plt.tight_layout()
plt.show()

In [ ]:
# Plot 
for time_step in range(1, num_steps + 1):
    # Load newcomer bid data for the current time step
    loaded_data = model_out[model_out['time_step'] == time_step]['newcomer_bid'].values[0]

    # Extract data
    newcomer_bids, dist_values, savings_values = zip(*loaded_data)

    # Create a 3D surface plot
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_trisurf(dist_values, savings_values, newcomer_bids, cmap='viridis', edgecolor='k')
    ax.scatter(dist_values, savings_values, newcomer_bids, marker='*', color='red', s=100, label='Newcomer Bids')
    ax.set_title(f'Newcomer Bids - Time Step {time_step}')
    ax.set_xlabel('Distance')
    ax.set_ylabel('Savings')
    ax.set_zlabel('Newcomer Bid')
    plt.show()

In [ ]:
print(model_out['R_N'][0])